In [2]:
import STAligner

# the location of R (used for the mclust clustering)
import os
# os.environ['R_HOME'] = "/data/kanferg/conda/envs/env_STAligner/lib"
# os.environ['R_USER'] = "/data/kanferg/conda/envs/env_STAligner/lib/python3.8/site-packages/rpy2"
# import rpy2.robjects as robjects
# import rpy2.robjects.numpy2ri

import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg

import torch
print(torch.cuda.device_count())

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


1


In [3]:
used_device = torch.device('cuda')

In [4]:
import scipy
import networkx

print("SciPy version:", scipy.__version__)
print("NetworkX version:", networkx.__version__)

SciPy version: 1.10.1
NetworkX version: 2.8.4


In [5]:
path =  "/data/kanferg/Sptial_Omics/playGround/Data/embryo_slices/"

In [6]:
Batch_list = []
adj_list = []

section_ids = ['E9.5_E1S1', 'E10.5_E1S1', 'E11.5_E1S1']
for section_id in section_ids:
    print(section_id)
    adata = sc.read_h5ad(os.path.join(path + section_id + ".MOSTA.h5ad"))
    adata.X = adata.layers['count']

    # make spot name unique
    adata.obs_names = [x + '_' + section_id for x in adata.obs_names]

    STAligner.Cal_Spatial_Net(adata, rad_cutoff=1.3)

    # Normalization
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata,  n_top_genes=5000) #ensure enough common HVGs in the combined matrix
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

E9.5_E1S1
------Calculating spatial graph...


/gpfs/gsfs10/users/kanferg/Sptial_Omics/playGround/STAligner/STAligner/ST_utils.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spatial_Net['Cell1'] = Spatial_Net['Cell1'].map(id_cell_trans)
/gpfs/gsfs10/users/kanferg/Sptial_Omics/playGround/STAligner/STAligner/ST_utils.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spatial_Net['Cell2'] = Spatial_Net['Cell2'].map(id_cell_trans)


The graph contains 23166 edges, 5913 cells.
3.9178 neighbors per cell on average.
E10.5_E1S1
------Calculating spatial graph...


/gpfs/gsfs10/users/kanferg/Sptial_Omics/playGround/STAligner/STAligner/ST_utils.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spatial_Net['Cell1'] = Spatial_Net['Cell1'].map(id_cell_trans)
/gpfs/gsfs10/users/kanferg/Sptial_Omics/playGround/STAligner/STAligner/ST_utils.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spatial_Net['Cell2'] = Spatial_Net['Cell2'].map(id_cell_trans)


The graph contains 72822 edges, 18408 cells.
3.9560 neighbors per cell on average.
E11.5_E1S1
------Calculating spatial graph...


/gpfs/gsfs10/users/kanferg/Sptial_Omics/playGround/STAligner/STAligner/ST_utils.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spatial_Net['Cell1'] = Spatial_Net['Cell1'].map(id_cell_trans)
/gpfs/gsfs10/users/kanferg/Sptial_Omics/playGround/STAligner/STAligner/ST_utils.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spatial_Net['Cell2'] = Spatial_Net['Cell2'].map(id_cell_trans)


The graph contains 119204 edges, 30124 cells.
3.9571 neighbors per cell on average.


In [7]:
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')
print('adata_concat.shape: ', adata_concat.shape)

adata_concat.shape:  (54445, 436)


In [8]:
np.asarray(adj_list[0].todense())

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [9]:
# adj_concat = np.asarray(adj_list[0].todense())
# for batch_id in range(1,len(section_ids)):
#     adj_concat = scipy.linalg.block_diag(adj_concat, np.asarray(adj_list[batch_id].todense()))
# adata_concat.uns['edgeList'] = np.nonzero(adj_concat)

In [10]:
adj_concat = np.asarray(adj_list[0].todense())
adj_concat = scipy.linalg.block_diag(adj_concat, np.asarray(adj_list[1].todense()))

In [11]:
adj_concat = scipy.linalg.block_diag(adj_concat, np.asarray(adj_list[2].todense()))

In [12]:
#adj_concat = scipy.linalg.block_diag(adj_concat, np.asarray(adj_list[3].todense()))

In [13]:
adata_concat.uns['edgeList'] = np.nonzero(adj_concat) 

In [14]:
iter_comb = [(1, 0), (2, 0)] ## Fix slice 0 as reference to align
adata_concat = STAligner.train_STAligner(adata_concat, verbose=True, knn_neigh = 100, iter_comb = iter_comb,
                                                        margin=2.5,  device=used_device)

STAligner(
  (conv1): GATConv(436, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 436, heads=1)
)
Pretrain with STAGATE...


100%|██████████████████████████████████████████████████████████| 500/500 [00:22<00:00, 21.90it/s]


Train with STAligner...


  0%|                                                                    | 0/500 [00:00<?, ?it/s]

Update spot triplets at epoch 500


...

 60%|██████████████████████████████████▊                       | 300/500 [01:02<00:09, 21.99it/s]

Update spot triplets at epoch 800


 80%|██████████████████████████████████████████████▍           | 400/500 [01:25<00:04, 21.99it/s]

Update spot triplets at epoch 900


100%|██████████████████████████████████████████████████████████| 500/500 [01:48<00:00,  4.59it/s]


In [21]:
adata_concat.uns

OverloadedDict, wrapping:
	OrderedDict([('edgeList', (array([    0,     0,     0, ..., 54444, 54444, 54444]), array([    0,     1,    28, ..., 54421, 54443, 54444])))])
With overloaded keys:
	['neighbors'].

In [23]:
adata_concat_temp = adata_concat.copy()

In [24]:
del adata_concat_temp.obs

In [28]:
adata_concat_temp.shape

(54445, 436)

In [27]:
adata_concat_temp.obsm['STAligner'].shape

(54445, 30)

In [ ]:
sc.pp.neighbors(adata_concat, use_rep='STAligner', random_state=666)
sc.tl.louvain(adata_concat, random_state=666, key_added="louvain", resolution=0.4)